In [ ]:
import pandas as pd
import spacy
from collections import Counter
import itertools
import re
import matplotlib.pyplot as plt
# Załadowanie modelu języka polskiego
nlp = spacy.load('pl_core_news_md')
anomaly_opinions = pd.read_csv('csv_data/preprocesed_files/anomaly_opinions.csv')
normal_opinions = pd.read_csv('csv_data/preprocesed_files/normal_opinions.csv')

In [ ]:


def unique_words_in_csv(file_path, column_name):
    lem_counter = Counter()

    df = pd.read_csv(file_path)
    content_column = df[column_name]

    for content in content_column:
        doc = nlp(content)

        for token in doc:
            if token.is_alpha and (not token.is_stop or token.lower_ == "niż" or token.lower_ == "od"):
                lem_counter[token.lemma_.lower()] += 1

    lemmas_count = list(zip(lem_counter.keys(), lem_counter.values()))
    sorted_lemmas_count = sorted(lemmas_count, key=lambda x: x[1], reverse=True)
    return sorted_lemmas_count


In [ ]:
# Przykładowe użycie
file_path = 'csv_data/preprocesed_files/anomaly_opinions.csv'
column_name = 'content'

lemmas_count = unique_words_in_csv(file_path, column_name)


In [ ]:
def print_lemmas_in_dataset(sorted_lemmas_count):
    # Przygotowanie danych do wykresu
    words = [lemma for lemma, count in sorted_lemmas_count if count > 5]
    counts = [count for lemma, count in sorted_lemmas_count if count > 5]

    # Generowanie wykresu słupkowego
    plt.figure(figsize=(10, 6))  # Zmiana rozmiaru wykresu
    plt.bar(range(len(words)), counts)
    plt.xticks(range(len(words)), words, rotation=45, ha='right')  # Obrót etykiet o 45 stopni
    plt.xlabel("Unikalne Słowa")
    plt.ylabel("Częstość występowania")
    plt.title("Częstość występowania unikalnych słów (częstość > 5)")
    plt.tight_layout()
    plt.show()

In [ ]:
print_lemmas_in_dataset(lemmas_count)

In [ ]:
# Inicjalizacja pustej listy
lemmas = []

# Iteracja po liście lemmas_count
for lemma, count in lemmas_count:
    if count > 5:
        lemmas.append(lemma)
len(lemmas)

In [ ]:

def create_combinations(manually_selected_lemmas, lemmas):
    combinations = set()

    # Generowanie wszystkich kombinacji bez duplikatów
    for r in range(1, len(manually_selected_lemmas) + 1):
        for combination in itertools.combinations(lemmas, r):
            combinations.add(tuple(sorted(manually_selected_lemmas + list(combination))))

    return combinations


In [ ]:
manually_selected_lemmas = ["niż", "lepszy", "lepiej", "rynek", "jakość", "znacznie", "od"]

combinations = create_combinations(manually_selected_lemmas, lemmas)

for combination in combinations:
    print(combination)

print("Liczba kombinacji:", len(combinations))

In [ ]:
def create_rules_from_lemmas(lemmas):
    rules = []

    # Tworzenie reguł na podstawie analizy częstych kombinacji
    for i in range(len(lemmas)):
        for j in range(i + 1, len(lemmas)):
            combination = (lemmas[i], lemmas[j])
            rule = ' '.join(combination)  # Tworzenie reguły przez połączenie słów

            rules.append(rule)  # Dodawanie reguły do zbioru

    return rules


rules = create_rules_from_lemmas(lemmas)

for rule in rules:
    print(rule)

len(rules)

In [ ]:
def check_dual_quality_opinion(zdanie):
    # Przetworzenie zdania na tokeny
    doc = nlp(zdanie)

    # Tworzenie listy lematów zdania
    #TU CHYBA TEŻTRZEBA DAĆ ZEBY NIE USUWAŁO NAM "NIŻ"
    lemmas = [token.lemma_.lower() for token in doc]

    # Przetworzenie lematów na zdanie
    processed_sentence = ' '.join(lemmas)

    # Sprawdzenie czy występują lematy słów kluczowych w zdaniu
    for rule in rules:
        if rule in processed_sentence:
            return True

    return False

In [ ]:

# Przykładowe zdania do sprawdzenia
zdanie1 = "Vizir jak Vizir. Trochę lepszy niż dostępny w polskich sklepach."
zdanie2 = "myję arielem ponieważ moja rodzina uwielbia zapach ariel wynik prania przekonuje mnie i jest również bardziej zrównoważony jako środek zaradczy dla mojej opinii niż detergent płynny lub kapsuły."
zdanie3 = "Nie ma różnicy między produktem w różnych krajach."

In [ ]:
# Sprawdzenie pierwszego zdania
if check_dual_quality_opinion(zdanie1):
    print("Zdanie 1 wskazuje na podwójną jakość produktu.")
else:
    print("Zdanie 1 nie wskazuje na podwójną jakość produktu.")

# Sprawdzenie drugiego zdania
if check_dual_quality_opinion(zdanie2):
    print("Zdanie 2 wskazuje na podwójną jakość produktu.")
else:
    print("Zdanie 2 nie wskazuje na podwójną jakość produktu.")

# Sprawdzenie trzeciego zdania
if check_dual_quality_opinion(zdanie3):
    print("Zdanie 3 wskazuje na podwójną jakość produktu.")
else:
    print("Zdanie 3 nie wskazuje na podwójną jakość produktu.")
